In [2]:
# Setup the SparkSession
from anovos.shared.spark import spark

# Import function for data loading
from anovos.data_ingest.data_ingest import read_dataset
from anovos.data_analyzer.stats_generator import measures_of_percentiles

# Import other modules
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [3]:
output_path = "../data/pre_saved_stats/"

## Read Data

In [4]:
df = read_dataset(
    spark,
    file_path='../data/application_train_modified.csv',
    file_type='csv',
    file_configs={'header': 'True', 'delimiter': ',', 'inferSchema': 'True'})

In [5]:
stability_cols = ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
                  'DAYS_EMPLOYED', 'DAYS_BIRTH', 'CNT_FAM_MEMBERS', 'CNT_CHILDREN',
                  'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']
cols_sample = ['SK_ID_CURR', 'TARGET', 'AMT_ANNUITY', 'AMT_CREDIT', 'AMT_GOODS_PRICE', 'AMT_INCOME_TOTAL', 
               'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_MON', 
               'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 
               'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'COMMONAREA_MEDI', 'FLAG_CONT_MOBILE', 'FLAG_PHONE',
               'CNT_CHILDREN', 'CNT_FAM_MEMBERS', 'CODE_GENDER', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 
               'DAYS_ID_PUBLISH', 'NAME_CONTRACT_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 
               'NAME_INCOME_TYPE', 'NAME_TYPE_SUITE', 'ORGANIZATION_TYPE', 'REGION_RATING_CLIENT_W_CITY', 
               'LIVE_CITY_NOT_WORK_CITY']

## Generate Historical Datasets for Data Stability

In [6]:
df = df.select(stability_cols+cols_sample).withColumn("id", F.monotonically_increasing_id())\
    .withColumn("id", F.row_number().over(Window.orderBy("id")))

In [7]:
from anovos.data_analyzer.stats_generator import measures_of_percentiles
odf = measures_of_percentiles(spark, idf = df)
odf.toPandas()

22/11/28 17:37:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:37:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/11/28 17:37:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:37:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/11/28 17:37:44 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,attribute,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
0,AMT_ANNUITY,1615.5000,6174.0000,9000.0000,11074.5000,16524.0000,24903.0000,34596.0000,4.595400e+04,5.332500e+04,7.003350e+04,2.580255e+05
1,AMT_CREDIT,45000.0000,76410.0000,135000.0000,180000.0000,270000.0000,513531.0000,808650.0000,1.133748e+06,1.350000e+06,1.847880e+06,4.050000e+06
2,AMT_GOODS_PRICE,40500.0000,67500.0000,135000.0000,180000.0000,238500.0000,450000.0000,679500.0000,1.093500e+06,1.305000e+06,1.800000e+06,4.050000e+06
3,AMT_INCOME_TOTAL,25650.0000,45000.0000,67500.0000,81000.0000,112500.0000,147150.0000,202500.0000,2.700000e+05,3.375000e+05,4.725000e+05,1.170000e+08
4,AMT_REQ_CREDIT_BUREAU_DAY,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+00
5,AMT_REQ_CREDIT_BUREAU_MON,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.000000e+00,1.000000e+00,4.000000e+00,2.700000e+01
6,APARTMENTS_MEDI,0.0000,0.0021,0.0083,0.0167,0.0583,0.0864,0.1489,2.425000e-01,3.300000e-01,5.496000e-01,1.000000e+00
7,BASEMENTAREA_MEDI,0.0000,0.0000,0.0000,0.0000,0.0437,0.0758,0.1116,1.678000e-01,2.230000e-01,3.835000e-01,1.000000e+00
8,CNT_CHILDREN,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,2.000000e+00,2.000000e+00,3.000000e+00,1.900000e+01
9,CNT_FAM_MEMBERS,1.0000,1.0000,1.0000,1.0000,2.0000,2.0000,3.0000,3.000000e+00,4.000000e+00,5.000000e+00,2.000000e+01


In [8]:
df1 = df.where(F.col('id')<=500).drop('id')
df2 = df.where((F.col('id')>2000) & (F.col('id')<=2500)).drop('id')
df3 = df.where((F.col('id')>3000) & (F.col('id')<=3500)).drop('id')
df4 = df.where((F.col('id')>4000) & (F.col('id')<=4500)).drop('id')
df5 = df.where((F.col('id')>5000) & (F.col('id')<=5500)).drop('id')
df6 = df.where((F.col('id')>6000) & (F.col('id')<=6500)).drop('id')

## Calculate Stability Index
The following code generate a dataset in **data/pre_saved_stats/stability**

In [9]:
from anovos.drift_stability.stability import stability_index_computation
df_stability = stability_index_computation(spark, [df1, df2, df3, df4, df5, df6], 
                                           appended_metric_path=output_path+'stability', threshold=2)
df_stability.toPandas()

22/11/28 17:37:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:37:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:37:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:37:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:37:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:37:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


/Users/zhuli/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


22/11/28 17:38:19 WARN DAGScheduler: Broadcasting large task binary with size 8.2 MiB


22/11/28 17:38:22 WARN DAGScheduler: Broadcasting large task binary with size 8.2 MiB


22/11/28 17:38:26 WARN DAGScheduler: Broadcasting large task binary with size 8.4 MiB


,attribute,type,mean_stddev,mean_cv,stddev_cv,kurtosis_cv,mean_si,stddev_si,kurtosis_si,stability_index,flagged
0,EXT_SOURCE_1,Numerical,0.0110,0.0218,0.0268,0.0465,4.0,4.0,3.0,3.8,0
1,DAYS_BIRTH,Numerical,254.4689,-0.0158,0.1445,1.0563,4.0,2.0,0.0,2.6,0
2,APARTMENTS_MEDI,Numerical,0.0056,0.0484,0.0916,0.5219,3.0,3.0,0.0,2.4,0
3,AMT_ANNUITY,Numerical,917.4112,0.0340,0.0879,0.2017,3.0,3.0,1.0,2.6,0
4,LIVE_CITY_NOT_WORK_CITY,Numerical,0.0162,0.0905,0.0354,0.1269,3.0,3.0,2.0,2.8,0
5,AMT_CREDIT,Numerical,21077.6419,0.0351,0.0593,0.1368,3.0,3.0,2.0,2.8,0
6,CNT_CHILDREN,Numerical,0.0194,0.0466,0.0504,0.5703,3.0,3.0,0.0,2.4,0
7,SK_ID_CURR,Numerical,2524.6105,0.0242,0.0223,0.0061,4.0,4.0,4.0,4.0,0
8,EXT_SOURCE_3,Numerical,0.0119,0.0230,0.0555,0.0917,4.0,3.0,3.0,3.5,0
9,FLAG_PHONE,Numerical,0.0192,0.0691,0.0207,0.1017,3.0,4.0,2.0,3.1,0


In [10]:
df_intermediate = read_dataset(spark, output_path+"stability", "csv", {'header':True,'inferSchema':True})
df_intermediate.show(10, False)

+---+-----------------------+---------+-------------------+-------------------+------------------+
|idx|attribute              |type     |mean               |stddev             |kurtosis          |
+---+-----------------------+---------+-------------------+-------------------+------------------+
|1  |EXT_SOURCE_1           |Numerical|0.5065364652985783 |0.2108052944623017 |2.147079910705088 |
|1  |DAYS_BIRTH             |Numerical|-16137.382         |5185.191343684167  |14.94182904807309 |
|1  |APARTMENTS_MEDI        |Numerical|0.11338651685393251|0.09943723020002199|8.126497681974076 |
|1  |AMT_ANNUITY            |Numerical|27020.943          |14513.443644776777 |6.614352737036268 |
|1  |LIVE_CITY_NOT_WORK_CITY|Numerical|0.204              |0.40337255588063414|3.158242191348906 |
|1  |AMT_CREDIT             |Numerical|595513.566         |401230.1596290132  |4.62314261396233  |
|1  |CNT_CHILDREN           |Numerical|0.408              |0.7201202304425179 |5.206881657740683 |
|1  |SK_ID

## Save Statistics for Full Report Generation

In [11]:
from anovos.data_ingest.data_ingest import select_column, write_dataset
report_stats = output_path + 'report_stats/'
df_sample = select_column(df.sample(False, 0.033, 13), list_of_cols=cols_sample)

The following code generate required data in **data/pre_saved_stats/report_stats** which will be used to generate the full report in use case demo.

In [13]:
# Save required statistics for drift detection
from anovos.drift_stability.drift_detector import statistics
from anovos.drift_stability.stability import stability_index_computation
import pyspark.sql.functions as F

source_df = select_column(df.where(F.col('TARGET')==1).sample(False, 0.08, 7), list_of_cols=cols_sample)
odf1 = statistics(spark, df_sample, source_df, drop_cols='SK_ID_CURR', method_type="all", source_path=report_stats)
write_dataset(odf1, report_stats+'drift_output/', 'parquet',{'mode':'overwrite'})

#Save required statistics for stability analysis
df7 = select_column(df.sample(False, 0.0065, 7), list_of_cols=cols_sample)
odf2 = stability_index_computation(spark, [df7], drop_cols='SK_ID_CURR',
                                   existing_metric_path=output_path+'stability', 
                                   appended_metric_path=report_stats+'stabilityIndex_metrics',
                                   threshold=2)
write_dataset(odf2, report_stats+'stability_index', 'parquet',{'mode':'overwrite'})

2022-11-28 17:41:54.429 | DEBUG    | anovos.drift_stability.validations:validate:20 - check the list of columns


22/11/28 17:41:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/11/28 17:42:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:42:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:42:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:42:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/11/28 17:43:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/11/28 17:43:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


...

22/11/28 17:43:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/11/28 17:43:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:43:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 1

22/11/28 17:44:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:44:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:44:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:44:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:44:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 17:44:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/11/28 1

22/11/28 17:44:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/11/28 17:44:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
